In [74]:
import numpy as np
import pandas as pd
import networkx as nx
from collections import Counter
from gensim.models import Word2Vec
import os
#import matplotlib.pyplot as plt

In [75]:
raw_songs_dataset = pd.read_csv("../Data/songs_dataset.csv")


raw_songs_dataset.head()

,song_id,song_title,year,release,tempo,loudness,duration,song_hotttnesss,artist_id,artist_name,artist_latitude,artist_longitude,artist_location,artist_hotttnesss,artist_familiarity
0,SOVFVAK12A8C1350D9,Tanssi vaan,1995.0,Karkuteillä,150.778,-10.555,156.55138,0.299877,ARMVN3U1187FB3A1EB,Karkkiautomaatti,NaN,NaN,NaN,0.356992,0.439604
1,SOGTUKN12AB017F4F1,No One Could Ever,2006.0,Butter,177.768,-2.060,138.97098,0.617871,ARGEKB01187FB50750,Hudson Mohawke,55.8578,-4.24251,"Glasgow, Scotland",0.437504,0.643681
2,SOBNYVR12A8C13558C,Si Vos Querés,2003.0,De Culo,87.433,-4.654,145.05751,NaN,ARNWYLR1187B9B2F9C,Yerba Brava,NaN,NaN,NaN,0.372349,0.448501
3,SOHSBXH12A8C13B0DF,Tangle Of Aspens,NaN,Rene Ablaze Presents Winter Sessions,140.035,-7.806,514.29832,NaN,AREQDTE1269FB37231,Der Mystic,NaN,NaN,NaN,0.000000,0.000000
4,SOZVAPQ12A8C13B63C,"Symphony No. 1 G minor ""Sinfonie Serieuse""/All...",NaN,Berwald: Symphonies Nos. 1/2/3/4,90.689,-21.420,816.53506,NaN,AR2NS5Y1187FB5879D,David Montgomery,NaN,NaN,NaN,0.109626,0.361287


In [41]:
artist_dataset = pd.read_csv("../Data/artist_terms.csv")

artist_dataset.head(27)

,artist_id,term
0,AR002UA1187B9A637D,garage rock
1,AR002UA1187B9A637D,country rock
2,AR002UA1187B9A637D,free jazz
3,AR002UA1187B9A637D,oi
4,AR002UA1187B9A637D,space rock
5,AR002UA1187B9A637D,power pop
6,AR002UA1187B9A637D,indie rock
7,AR002UA1187B9A637D,reggae
8,AR002UA1187B9A637D,pop rock
9,AR002UA1187B9A637D,rock


In [71]:

new_artist_dataset = []

# Load pre-trained Word2Vec model
word2vec_model = Word2Vec.load(os.path.join('../models', 'imdb_w2v.model'))

# Function to get vector representation of a genre
def get_genre_vector(genre):
    try:
        return word2vec_model.wv[genre]
    except KeyError:
        return np.zeros(word2vec_model.vector_size)  # Return zero vector if genre not in vocabulary

# Function to calculate average vector for a list of genre vectors
def calculate_average_vector(genre_vectors):
    return np.mean(genre_vectors, axis=0)

# Function to convert vector back to word
def vector_to_word(vector):
    return word2vec_model.wv.similar_by_vector(vector, topn=1)[0][0]

for artist_id,group in artist_dataset.groupby("artist_id"):
    
    # 1. Convert each genre word into a vector for artist_id and store it in genres_vectors_for_artist_id
    genres_vectors_for_artist_id = [get_genre_vector(genre) for genre in group["term"]]
    
    # 2. Find the average vector for genres_vectors_for_artist_id
    avg_vec = calculate_average_vector(genres_vectors_for_artist_id)
    
    # 3. Convert avg_vec back to a word
    avg_genre_word = vector_to_word(avg_vec)
    
    new_artist_dataset.append({
        "artist_id": artist_id,
        "term" : avg_genre_word
    })
    

new_artist_dataset = pd.DataFrame(new_artist_dataset)
new_artist_dataset.head()


,artist_id,term
0,AR002UA1187B9A637D,grunge
1,AR003FB1187B994355,ramones
2,AR006821187FB5192B,motown
3,AR009211187B989185,grunge
4,AR009SZ1187B9A73F4,rhythms


In [73]:
new_artist_dataset.to_csv("../Data/new_artist_dataset.csv",index=False)

In [76]:
merged_dataset = pd.merge(raw_songs_dataset, new_artist_dataset, on='artist_id', how='left')

In [80]:
merged_dataset.to_csv("../Data/merged_dataset.csv",index=False)

In [3]:
raw_songs_dataset.isna().sum()

song_id                    0
song_title                 2
year                  484270
release                    7
tempo                      0
loudness                   0
duration                   0
song_hotttnesss       417782
artist_id                  0
artist_name                0
artist_latitude       641766
artist_longitude      641766
artist_location       487546
artist_hotttnesss         12
artist_familiarity       185
dtype: int64

In [4]:
raw_songs_dataset.isna().sum().sum()

2673336

Shartil: for now I am going to delete all rows with missing data.<br>
This is an initial approach, let's discuss it together with Elisa.

In [5]:
songs_dataset = raw_songs_dataset.dropna()

songs_dataset.head()

,song_id,song_title,year,release,tempo,loudness,duration,song_hotttnesss,artist_id,artist_name,artist_latitude,artist_longitude,artist_location,artist_hotttnesss,artist_familiarity
1,SOGTUKN12AB017F4F1,No One Could Ever,2006.0,Butter,177.768,-2.060,138.97098,0.617871,ARGEKB01187FB50750,Hudson Mohawke,55.85780,-4.24251,"Glasgow, Scotland",0.437504,0.643681
14,SOSDCFG12AB0184647,006,1998.0,Lena 20 År,122.332,-3.925,262.26893,0.212045,ARSB5591187B99A848,Lena Philipsson,57.42635,15.08518,"Vetlanda, Sweden",0.410229,0.529819
15,SOBARPM12A8C133DFF,(Looking For) The Heart Of Saturday,1994.0,Cover Girl,99.214,-14.379,216.47628,0.270776,ARDW5AW1187FB55708,Shawn Colvin,42.78668,-96.92803,"Vermillion, SD",0.446733,0.685503
16,SOKOVRQ12A8C142811,Ethos of Coercion,2009.0,Descend Into Depravity,189.346,-6.366,196.02240,0.614766,ARGWPP11187B9AEF43,Dying Fetus,38.82350,-75.92381,Maryland,0.511976,0.734471
32,SOOLRHW12A8C142643,All of the same blood,2001.0,Violent revolution,191.665,-6.663,372.42730,0.788727,AR79L0D1187FB3AFB6,Kreator,51.45180,7.01062,Essen,0.472691,0.740252


In [6]:
len(songs_dataset)

126910

Shartil: I am adding new column called "decade".

In [7]:
YEAR_COLUMN_NAME = "year"
DECADE_COLUMN_NAME = "decade"
SONG_TITLE_COLUMN_NAME = "song_title"

In [8]:
songs_dataset = songs_dataset.assign(decade=lambda row: (row[YEAR_COLUMN_NAME].astype(int) // 10) * 10)

songs_dataset.head()

,song_id,song_title,year,release,tempo,loudness,duration,song_hotttnesss,artist_id,artist_name,artist_latitude,artist_longitude,artist_location,artist_hotttnesss,artist_familiarity,decade
1,SOGTUKN12AB017F4F1,No One Could Ever,2006.0,Butter,177.768,-2.060,138.97098,0.617871,ARGEKB01187FB50750,Hudson Mohawke,55.85780,-4.24251,"Glasgow, Scotland",0.437504,0.643681,2000
14,SOSDCFG12AB0184647,006,1998.0,Lena 20 År,122.332,-3.925,262.26893,0.212045,ARSB5591187B99A848,Lena Philipsson,57.42635,15.08518,"Vetlanda, Sweden",0.410229,0.529819,1990
15,SOBARPM12A8C133DFF,(Looking For) The Heart Of Saturday,1994.0,Cover Girl,99.214,-14.379,216.47628,0.270776,ARDW5AW1187FB55708,Shawn Colvin,42.78668,-96.92803,"Vermillion, SD",0.446733,0.685503,1990
16,SOKOVRQ12A8C142811,Ethos of Coercion,2009.0,Descend Into Depravity,189.346,-6.366,196.02240,0.614766,ARGWPP11187B9AEF43,Dying Fetus,38.82350,-75.92381,Maryland,0.511976,0.734471,2000
32,SOOLRHW12A8C142643,All of the same blood,2001.0,Violent revolution,191.665,-6.663,372.42730,0.788727,AR79L0D1187FB3AFB6,Kreator,51.45180,7.01062,Essen,0.472691,0.740252,2000


In [9]:
min_decade = songs_dataset[DECADE_COLUMN_NAME].min()
max_decade = songs_dataset[DECADE_COLUMN_NAME].max()

decade_array = np.linspace(min_decade, max_decade, 10, dtype=int)

Shartil: now I am going to create the graph

In [10]:
decade_graph = nx.DiGraph()

In [11]:
decade_graph.add_nodes_from(decade_array.tolist())

decade_graph.add_nodes_from(songs_dataset[SONG_TITLE_COLUMN_NAME].tolist())

In [12]:
relationships = []
for index, row in songs_dataset.iterrows():
    current_song_title = row[SONG_TITLE_COLUMN_NAME]
    current_decade = row[DECADE_COLUMN_NAME]

    relationships.append((current_decade, current_song_title, {"label": "release_decade"}))

decade_graph.add_edges_from(relationships)

In [18]:
print(decade_graph)

DiGraph with 107333 nodes and 115512 edges


In [17]:
decade_input = 1920

songs_from_given_decade = [ song for song in decade_graph[decade_input].keys()]
songs_from_given_decade

['Big Fat Mama Blues',
 'Keep It Home',
 'Jungle Man Blues',
 "I'm A Mighty Tight Woman",
 'Big Bill Blues',
 'Blue Ghost Blues',
 "Ain't Misbehavin'",
 'Fat Mouth Blues',
 'Wreck Of The Old 97',
 "The Prisoner's Song",
 "Mother's Children Have A Hard Time",
 'Shake That Thing',
 'Canned Heat Blues',
 "Coffee Grindin' Blues",
 "That's No Way To Get Along",
 "Bessie's Moan"]